In [1]:
import requests
import shutil
import os
import io
import json
import time
import datetime
import dateutil
import random
import re
import ast
import pickle
import numpy as np
import pandas as pd
from IPython.display import clear_output
from pprint import pprint
import bs4
import matplotlib.pyplot as plt
import copy

parent_folder='steam_data'

csv_folder=os.path.join(parent_folder,"bulk","csv_no_text","early_access_standardized")
csv_english_folder=os.path.join(parent_folder,"bulk","csv_english","early_access_standardized")
testing_folder=os.path.join(parent_folder,"bulk","testing")

In [14]:
num_reviews_df=pd.read_csv(os.path.join(testing_folder,'20190126_num_reviews.csv')).set_index('appid')
early_access_start_time_df=pd.read_csv(os.path.join(testing_folder,'20190126_early_access_start_time.csv')).set_index('appid')
normal_review_start_time_df=pd.read_csv(os.path.join(testing_folder,'20190126_normal_review_start_time.csv')).set_index('appid')
apps_df=pd.read_csv(os.path.join(parent_folder,'bulk','feature_vecs','appwarnings.csv')).set_index('appid')
norm_game_vecs_df=pd.read_csv(os.path.join(parent_folder,'bulk','feature_vecs','catnormalized.csv')).set_index('appid')
col_names=norm_game_vecs_df.columns.tolist()
for i in col_names:
    norm_game_vecs_df[i+i]=norm_game_vecs_df[i]*norm_game_vecs_df[i]
norm_game_vecs_df['one']=1

In [15]:
review_folder=os.path.join(parent_folder,"bulk","csv_no_text","first_month_reviews")

num_reviews=[]
for i,appid in enumerate(apps_df.index.tolist(),1):
    #print(appid)
    month_df=pd.read_csv(os.path.join(review_folder,str(appid)+'_reviews_all.csv'))
    num_reviews.append(len(month_df))
    #if i>10:
    #    break
    clear_output(wait=True)
    print(str(i)+"/"+str(len(apps_df)))
apps_df['first_month_num_reviews']=num_reviews

4518/4518


In [17]:
apps_df.to_csv(os.path.join(parent_folder,'bulk','apps.csv'))

In [123]:
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import pandas as pd
import os

parent_folder='steam_data'

final_genres_list=['2D','4X','Survival','Open World','Adventure','Action','RPG','Indie','Dungeon Crawler','Casual','Strategy',
        'Simulation','Sandbox','Shooter','Horror','Sports','Visual Novel','First-Person',
        'Top-Down','Third Person','God Game','Rhythm',"Beat 'em up",'Survival Horror','Battle Royale',
        'Puzzle','PvP','Rogue-like','Tactics','Turn-Based','Real-Time','MOBA','Building','VR Only',
        'Platformer','Walking Simulator','Stealth','Side Scroller','Education','Metroidvania','Space Sim',
        'Management','Fighting','Hack and Slash','Flight','Exploration','Isometric','Runner',
        "Shoot 'Em Up",'Arcade','Racing','Board Game','Card Game','Software']
final_themes_list=['Western','Sci-fi','Anime','Realistic','Cyberpunk', 'Historical','Cute', 'Lovecraftian',
                   'Space','Relaxing', 'Magic','Retro','Atmospheric','Fantasy','Science','Steampunk',
                   'Zombies','Futuristic','Minimalist','Funny', 'Post-apocalyptic','Dinosaurs','Medieval',
                   'Tactical','Robots','Military','Naval','Colorful','War','Music']
final_features_list=['Pixel Graphics','Level Editor','Physics', 'Team-Based','Co-op','Multiplayer',
          'Massively Multiplayer','VR','Difficult','Procedural Generation',
          'Character Customization','Controller','Resource Management','Competitive',
          'Rogue-lite','Story Rich','Fast-Paced','Crafting','Split Screen',
                     'Downloadable Content', 'In-App Purchases']
final_genres_list=sorted(final_genres_list)
final_themes_list=sorted(final_themes_list)
final_features_list=sorted(final_features_list)

num_genres=len(final_genres_list)
num_themes=len(final_themes_list)
num_features=len(final_features_list)
themes_start=num_genres
features_start=num_genres+num_themes
themes_end=features_start
features_end=features_start+num_features

labels=final_genres_list+final_themes_list+final_features_list

apps_df=pd.read_csv(os.path.join(parent_folder,'bulk','apps.csv')).set_index('appid')
norm_game_vecs_df=pd.read_csv(os.path.join(parent_folder,'bulk','feature_vecs','catnormalized.csv')).set_index('appid')
training_vecs_df=pd.DataFrame.copy(norm_game_vecs_df)
col_names=training_vecs_df.columns.tolist()
for i in col_names:
    training_vecs_df[i+i]=training_vecs_df[i]*training_vecs_df[i]
training_vecs_df['one']=1
training_vecs=training_vecs_df.values
training_labels=apps_df['first_month_num_reviews'].values
regressor=RandomForestRegressor(n_estimators=128,n_jobs=-1,max_features= 'sqrt',random_state=613)
regressor.fit(training_vecs,training_labels)

#predicted_labels = regressor.predict(training_vecs[0:5,:])
#print(predicted_labels)
#print(np.argmax(predicted_labels))

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=128, n_jobs=-1,
           oob_score=False, random_state=613, verbose=0, warm_start=False)

In [127]:
one_vec=np.zeros(105)
form_vars=[1,60,62,5,7,100,101,30,35]

used_vars=[x for x in form_vars if x!=-1]
for x in used_vars:
    one_vec[x]=1

def split_vec_norm(x,idx_start,idx_end):
    if np.linalg.norm(x[idx_start:idx_end])!=0:
        x[idx_start:idx_end]=x[idx_start:idx_end]/np.linalg.norm(x[idx_start:idx_end])
    return x

def normalized_vec(x):
    x=split_vec_norm(x,0,num_genres)
    x=split_vec_norm(x,themes_start,themes_end)
    x=split_vec_norm(x,features_start,features_end)
    return x

def false_normalized_vec(x,true_normal_vec):
    if np.sum(true_normal_vec[:themes_start])!=1:
        x=split_vec_norm(x,0,num_genres)
    if np.sum(true_normal_vec[themes_start:themes_end])!=1:
        x=split_vec_norm(x,themes_start,themes_end)
    if np.sum(true_normal_vec[features_start:])!=1:
        x=split_vec_norm(x,features_start,features_end)
    return x

def perturbed_vec(x,b,used_vars,amt):
    v=np.copy(x)
    for i in range(0,9):
        if 2**i & b:
            v[used_vars[i]]+=amt
    return v

p_vecs=[]
if len(used_vars)<=1:
    print("why would you EVER do this")
else:
    true_normal_vec=np.copy(one_vec)
    true_normal_vec=normalized_vec(true_normal_vec)
    false_normal_vec=np.copy(true_normal_vec)
    if np.sum(true_normal_vec[:themes_start])==1:
        false_normal_vec[:themes_start]*=0.8
    if np.sum(true_normal_vec[themes_start:themes_end])==1:
        false_normal_vec[themes_start:themes_end]*=0.8
    if np.sum(true_normal_vec[features_start:])==1:
        false_normal_vec[features_start:]*=0.8
    
    
    bigger=[]
    smaller=[]
    round=0
    while len(bigger)==0 and len(smaller)==0 and round<4:
        round+=1
        
        for i in range(2**len(used_vars)-1):
            temp_v=perturbed_vec(false_normal_vec,i,used_vars,0.05*round)
            p_vecs.append(false_normalized_vec(temp_v,true_normal_vec))

        #for x in np.vstack(p_vecs):
        #    print(x)


        p_df=pd.DataFrame(np.vstack(p_vecs),columns=norm_game_vecs_df.columns)
        col_names=p_df.columns.tolist()
        for i in col_names:
            p_df[i+i]=p_df[i]*p_df[i]
        p_df['one']=1

        predicted=regressor.predict(p_df.values)

        max_idx=np.argmax(predicted)
        for x in used_vars:
            if p_vecs[max_idx][x]>false_normal_vec[x]:
                bigger.append(labels[x])
            elif p_vecs[max_idx][x]<false_normal_vec[x]:
                smaller.append(labels[x])
    if len(bigger)==0 and len(smaller)==0:
        output_string="We couldn't find a way to suggest a focus for your idea! Your idea may be unusual, or your input may not be descriptive enough to generate a response (try adding more tags)."
    else:
        if len(bigger)>0:
            output_string="We suggest emphasizing on the following aspect(s) of your idea at launch to increase its appeal to the Steam userbase:\n"+bigger[0]
            if len(bigger)>1:
                for i in range(1,len(bigger)-1):
                    output_string+=", "+bigger[i]
                for i in range(len(bigger)-1,len(bigger)-2):
                    output_string+=" and "+bigger[-1]
            if len(smaller)>0:
                output_string+="\n\nThe following aspect(s) of your idea may be less important:\n"+smaller[0]
                if len(smaller)>1:
                    for i in range(1,len(smaller)-1):
                        output_string+=", "+smaller[i]
                    for i in range(len(smaller)-1,len(smaller)-2):
                        output_string+=" and "+smaller[-1]
        elif len(smaller)>0:
            output_string="We suggest de-emphasizing the following aspect(s) of your idea at launch to increase its appeal to the Steam userbase:\n"+smaller[0]
            if len(smaller)>1:
                for i in range(1,len(smaller)-1):
                    output_string+=", "+smaller[i]
                for i in range(len(smaller)-1,len(smaller)-2):
                    output_string+=" and "+smaller[-1]
                
    print(output_string)
    

We suggest emphasizing on the following aspect(s) of your idea at launch to increase its appeal to the Steam userbase:
Fantasy, Battle Royale

The following aspect(s) of your idea may be less important:
4X, Futuristic, Board Game, Split Screen, Racing


In [141]:
apps_df['log']=apps_df['first_month_num_reviews'].apply(lambda x: max(1,np.log(x)))
apps_df['sqrtlog']=apps_df['first_month_num_reviews'].apply(lambda x: max(1,np.sqrt(np.log(x))))
dot_df=norm_game_vecs_df.dot(true_normal_vec).sort_values(ascending = False)
print(dot_df.index.tolist())
dot_df.multiply(apps_df['log']).sort_values(ascending = False).index.tolist()
dot_df.multiply(apps_df['sqrtlog']).sort_values(ascending = False).index.tolist()

[815290, 402310, 444670, 496290, 427910, 396900, 527230, 606230, 265970, 924020, 375480, 340580, 473770, 406220, 330270, 250560, 452060, 339860, 813820, 508410, 648050, 233610, 922630, 326160, 224500, 435150, 248390, 562230, 400570, 563390, 878110, 371510, 400450, 838350, 319910, 501950, 893870, 698640, 492150, 266510, 922080, 488440, 316080, 382050, 378720, 338040, 312430, 600630, 646570, 344890, 796340, 501220, 621070, 389670, 329970, 290340, 289130, 725960, 547860, 487000, 834910, 440900, 305960, 379210, 459760, 525350, 773430, 317440, 374040, 828730, 397060, 920690, 293780, 770100, 268360, 422590, 744040, 710250, 266190, 697010, 436560, 335210, 298560, 436510, 436480, 469600, 442660, 727460, 677710, 457790, 527990, 619340, 880770, 862490, 528580, 268670, 294020, 576480, 906660, 489520, 577470, 438920, 809180, 614260, 389530, 571860, 296570, 920680, 826160, 310510, 509580, 719180, 496660, 345180, 469710, 386930, 342940, 657810, 602480, 598060, 748820, 535270, 566270, 946610, 853580,

[402310,
 813820,
 527230,
 396900,
 606230,
 427910,
 838350,
 834910,
 473770,
 265970,
 815290,
 440900,
 496290,
 435150,
 233610,
 330270,
 444670,
 488440,
 924020,
 922630,
 547860,
 374040,
 646570,
 248390,
 319910,
 326160,
 340580,
 452060,
 250560,
 529180,
 469600,
 339280,
 681660,
 289130,
 489520,
 262060,
 627690,
 920690,
 444090,
 378720,
 305960,
 698640,
 492150,
 224500,
 397060,
 562230,
 429790,
 293780,
 266510,
 509580,
 400450,
 307880,
 248630,
 436560,
 342940,
 487120,
 878110,
 329970,
 375480,
 253250,
 648050,
 893870,
 379210,
 508410,
 841770,
 328760,
 351910,
 332310,
 782570,
 230230,
 621070,
 598060,
 312430,
 389530,
 345180,
 764050,
 697010,
 487000,
 290340,
 796340,
 389670,
 371510,
 512900,
 431450,
 718650,
 319050,
 298560,
 773430,
 922080,
 944120,
 449960,
 725960,
 464620,
 501950,
 442660,
 236110,
 920680,
 286160,
 316080,
 296570,
 372800,
 268360,
 488350,
 946610,
 300380,
 382050,
 528580,
 362490,
 770100,
 289520,
 383230,
 

In [144]:
print(apps_df.loc[815290]['content_warning'])

False


In [59]:
2**2-1

3